In [1]:
%%capture
!pip install langchain huggingface_hub transformers accelerate langchain-groq langchain_community
!pip install langchain-huggingface langchain_openai
!pip install python-dotenv

!pip install ipywidgets widgetsnbextension pandas-profiling

Installation took around 5min

Load  configuration from environment variables

In [2]:
import os
from dotenv import dotenv_values
config = dotenv_values(".env")
os.environ["GROQ_API_KEY"] = config["GROQ_API_KEY"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = config["HUGGINGFACEHUB_API_TOKEN"]
#print(config)
#print(os.environ)

 **A few words on LLM models**:

 -Mistral-7B-v0.1: This is a base language model from Mistral AI with 7 billion parameters, designed for general language understanding tasks without any specialized fine-tuning.

-Mistral-7B-Instruct-v0.2: An instruction-tuned variant of Mistral-7B, fine-tuned on a range of instruction-following datasets to better perform on tasks where following explicit instructions is required.

-Mixtral-8x7b-32768: A composite model made up of eight Mistral-7B models, optimized for parallel processing on hardware like the GroqChip, suitable for large-scale inference with a focus on performance and efficiency.

-LLaMA3-8B-8192: A third-generation LLaMA model with 8 billion parameters, designed for more efficient context window management and capable of handling longer sequences of up to 8,192 tokens.

In [3]:
hg_base_model_id     = "mistralai/Mistral-7B-v0.1"
hg_instruct_model_id = "mistralai/Mistral-7B-Instruct-v0.3" #"mistralai/Mistral-7B-Instruct-v0.2"

groq_model_mixtral   = "mixtral-8x7b-32768"
groq_model_llama3    = "llama3-8b-8192"



🔁 **Workflow Overview**:
   - Select model type (LLM or Chat) for your task.
   - Design a prompt to guide the model.
   - Run input through your chosen model.
   - Use Output Parser for neat results.

In [ ]:
#from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
   
llm = HuggingFaceEndpoint(repo_id=hg_instruct_model_id)
#Logic question
_question= "Question:Bob is in the living room. He walks to the kitchen, carrying a cup. He puts a ball in the cup and carries the cup to the bedroom. He turns the cup upside down, then walks to the garden. He puts the cup down in the garden, then walks to the garage. Where is the ball?"
print(llm.invoke(_question))

In [ ]:
print("mistral based model")
print("-------------------")
llm = HuggingFaceEndpoint(repo_id=hg_base_model_id)
print(llm.invoke(_question))

Lets look at grok responses

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage,SystemMessage

llm_groq = ChatGroq(model=groq_model_llama3)

messages = [
    SystemMessage(content="When I ask for help to write something, you will reply with a document that contains at least one joke"),
    HumanMessage(content="write a cpp hello world.")
]

response = llm_groq.invoke(messages)
print(type(response))
print(response.content)


# Output parsers

- Output parsers shape the AI's text output into a more usable form, like a database entry or a JSON object.

**Main Uses:**

1. They turn a block of text into organized data.
2. They can guide the AI on how to format its responses for consistency and ease of use.


In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.output_parsers.list import ListOutputParser

Without parsing output

In [ ]:

prompt = PromptTemplate(
    template="List 3 {things}",
    input_variables=["things"])

response = llm_groq.invoke(input=prompt.format(things="sports that don't use balls"))
print(response.content)

Instantiate output parser

In [ ]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Now let's see how to use the parsers instructions in the prompt.

In [ ]:
prompt = PromptTemplate(
    template="List 3 {things}.\n {format_instructions} reply only results",
    input_variables=["things"],
    partial_variables={"format_instructions": format_instructions})

output = llm_groq.predict(text=prompt.format(things="sports that don't use balls"))
print(type(output))
print(output)


Finally, we can parse the output to a list (Python object)

In [ ]:
output_parser.parse(output)

**LangChain Expression Language (LCEL) Overview**:
   - **Purpose**: Simplify building complex chains from basic components.
   - **Features**: Supports streaming, parallelism, and logging.

### 🛠️ Basic Use Case: Prompt + Model + Output Parser
   - **Common Approach**: Link a prompt template with a model.
   - **Chain Mechanism**: Using the `|` symbol, like a Unix pipe, to connect components.
   - **Process Flow**: User input → Prompt Template → Model → Output Parser.

In [15]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)


model = ChatGroq(model=groq_model_llama3,temperature=0)

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"subject": "ice cream flavors"})